# All the Imports

In [1]:
import sys
import os
import logging
sys.path.insert(1, '../src/')

In [2]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from lassonet import LassoNetClassifier


from RankCorr.picturedRocks import Rocks
import smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


CPU times: user 7.8 s, sys: 653 ms, total: 8.45 s
Wall time: 7.86 s


In [3]:
from os import path

In [4]:
model_save_path = '../data/{}/models/'
num_times = 10

  

In [5]:
data_locations = [
                  'CITE-seq',
                 # 'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel'
                 # 'synthetic'
]

In [6]:
result_axis = [
 ('All Markers', 'all_markers_results_{}.npy', 'all_markers_knn_results_{}.npy'), 
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('MarkerMap Unsupervised', 'marker_map_unsupervised_results_{}.npy', 'marker_map_unsupervised_knn_results_{}.npy'),
 ('MarkerMap Supervised', 'marker_map_supervised_results_{}.npy', 'marker_map_supervised_knn_results_{}.npy'),
 ('MarkerMap Mixed Supervision', 'marker_map_mixed_results_{}.npy', 'marker_map_mixed_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy'),
 ('LassoNet', 'lasso_net_results_{}.npy', 'lasso_net_knn_results_{}.npy')
]

## Mean Accuracies and F1

### Random Forest

In [7]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.076, 0.901)","(0.132, 0.83)","(0.07, 0.91)","(0.063, 0.922)","(0.066, 0.916)","(0.132, 0.83)","(0.091, 0.885)","(0.063, 0.924)","(0.065, 0.92)","(0.127, 0.836)","(0.062, 0.924)"
mouse_brain_broad,"(0.009, 0.991)","(0.163, 0.827)","(0.016, 0.984)","(0.013, 0.987)","(0.031, 0.969)","(0.089, 0.906)","(0.013, 0.987)","(0.006, 0.994)","(0.031, 0.969)","(0.126, 0.865)","(0.008, 0.991)"
paul,"(0.184, 0.796)","(0.531, 0.406)","(0.113, 0.885)","(0.161, 0.823)","(0.146, 0.846)","(0.382, 0.573)","(0.139, 0.856)","(0.108, 0.889)","(0.138, 0.857)","(0.398, 0.555)","(0.126, 0.868)"
zeisel,"(0.042, 0.957)","(0.192, 0.794)","(0.054, 0.945)","(0.07, 0.928)","(0.05, 0.95)","(0.187, 0.801)","(0.124, 0.868)","(0.045, 0.955)","(0.066, 0.931)","(0.2, 0.785)","(0.054, 0.945)"


### KNN

In [8]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.098, 0.898)","(0.191, 0.782)","(0.071, 0.912)","(0.066, 0.925)","(0.073, 0.913)","(0.18, 0.796)","(0.104, 0.885)","(0.062, 0.929)","(0.07, 0.92)","(0.18, 0.795)","(0.064, 0.926)"
mouse_brain_broad,"(0.103, 0.885)","(0.234, 0.744)","(0.027, 0.973)","(0.013, 0.987)","(0.041, 0.958)","(0.14, 0.848)","(0.015, 0.985)","(0.006, 0.994)","(0.041, 0.958)","(0.198, 0.782)","(0.009, 0.991)"
paul,"(0.321, 0.608)","(0.632, 0.375)","(0.222, 0.759)","(0.269, 0.704)","(0.245, 0.741)","(0.447, 0.521)","(0.268, 0.721)","(0.203, 0.789)","(0.253, 0.735)","(0.461, 0.513)","(0.228, 0.761)"
zeisel,"(0.054, 0.945)","(0.261, 0.726)","(0.054, 0.945)","(0.07, 0.929)","(0.054, 0.945)","(0.273, 0.717)","(0.153, 0.84)","(0.038, 0.961)","(0.067, 0.93)","(0.286, 0.7)","(0.059, 0.938)"


## Variances

### Random Forest

In [68]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 15)
    f1 = round(f1, 15)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.006990333290383, 0.007214134999314)","(0.035409671167187, 0.040280393656022)","(0.002495840266223, 0.002588876583652)","(0.0, 0.0)","(0.00946519976927, 0.009624897773903)","(0.045678043992489, 0.049285721054077)","(0.048104654907401, 0.05401823739169)","(0.006261193917546, 0.006478326264052)","(0.010784546038668, 0.01218630903284)","(0.044230783929963, 0.051432224503959)","(0.009552545819905, 0.009544864725478)"
mouse_brain_broad,"(0.006990333290383, 0.007214134999314)","(0.035409671167187, 0.040280393656022)","(0.002495840266223, 0.002588876583652)","(0.0, 0.0)","(0.00946519976927, 0.009624897773903)","(0.045678043992489, 0.049285721054077)","(0.048104654907401, 0.05401823739169)","(0.006261193917546, 0.006478326264052)","(0.010784546038668, 0.01218630903284)","(0.044230783929963, 0.051432224503959)","(0.009552545819905, 0.009544864725478)"
paul,"(0.006990333290383, 0.007214134999314)","(0.035409671167187, 0.040280393656022)","(0.002495840266223, 0.002588876583652)","(0.0, 0.0)","(0.00946519976927, 0.009624897773903)","(0.045678043992489, 0.049285721054077)","(0.048104654907401, 0.05401823739169)","(0.006261193917546, 0.006478326264052)","(0.010784546038668, 0.01218630903284)","(0.044230783929963, 0.051432224503959)","(0.009552545819905, 0.009544864725478)"
zeisel,"(0.006990333290383, 0.007214134999314)","(0.035409671167187, 0.040280393656022)","(0.002495840266223, 0.002588876583652)","(0.0, 0.0)","(0.00946519976927, 0.009624897773903)","(0.045678043992489, 0.049285721054077)","(0.048104654907401, 0.05401823739169)","(0.006261193917546, 0.006478326264052)","(0.010784546038668, 0.01218630903284)","(0.044230783929963, 0.051432224503959)","(0.009552545819905, 0.009544864725478)"


### KNN

In [67]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 15)
    f1 = round(f1, 15)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.00825935018036, 0.008494210005314)","(0.035131802366684, 0.036415671495428)","(0.001996672212978, 0.002073402621801)","(0.0, 0.0)","(0.009270141948209, 0.009687345155385)","(0.056097892729881, 0.061257961883791)","(0.04963871510874, 0.05202863732319)","(0.004553721192481, 0.004651296218417)","(0.0119765663314, 0.012631493984938)","(0.044263633106143, 0.049575326771285)","(0.01007308543605, 0.01166240504249)"
mouse_brain_broad,"(0.00825935018036, 0.008494210005314)","(0.035131802366684, 0.036415671495428)","(0.001996672212978, 0.002073402621801)","(0.0, 0.0)","(0.009270141948209, 0.009687345155385)","(0.056097892729881, 0.061257961883791)","(0.04963871510874, 0.05202863732319)","(0.004553721192481, 0.004651296218417)","(0.0119765663314, 0.012631493984938)","(0.044263633106143, 0.049575326771285)","(0.01007308543605, 0.01166240504249)"
paul,"(0.00825935018036, 0.008494210005314)","(0.035131802366684, 0.036415671495428)","(0.001996672212978, 0.002073402621801)","(0.0, 0.0)","(0.009270141948209, 0.009687345155385)","(0.056097892729881, 0.061257961883791)","(0.04963871510874, 0.05202863732319)","(0.004553721192481, 0.004651296218417)","(0.0119765663314, 0.012631493984938)","(0.044263633106143, 0.049575326771285)","(0.01007308543605, 0.01166240504249)"
zeisel,"(0.00825935018036, 0.008494210005314)","(0.035131802366684, 0.036415671495428)","(0.001996672212978, 0.002073402621801)","(0.0, 0.0)","(0.009270141948209, 0.009687345155385)","(0.056097892729881, 0.061257961883791)","(0.04963871510874, 0.05202863732319)","(0.004553721192481, 0.004651296218417)","(0.0119765663314, 0.012631493984938)","(0.044263633106143, 0.049575326771285)","(0.01007308543605, 0.01166240504249)"


In [119]:
indices =  np.load('../data/paul/models/experiment_data_folds/rankcorr_indices_7.npy', allow_pickle = True)

In [120]:
indices2 =  np.load('../data/paul/models/experiment_data_folds/rankcorr_indices_9.npy', allow_pickle = True)

In [121]:
indices[1]

array([ 304, 2705, 1875, 1094, 2581,  943, 2628,  768,  957,  368, 1016,
       1263, 2137, 1248,  384,  767,  331, 1621,   29,  282, 1384, 2719,
        125, 1344,  358,  973,  244,   65, 1697, 1681,  609, 2069, 2351,
       2117, 1492, 1355, 2361, 1425, 1220,  123, 1930, 1980, 2380, 1528,
       2439, 1275, 1157,  762, 1979,  716, 1958, 2433, 1653,  404, 1688,
       2209,  481, 1289, 2253, 2136,  953, 2514, 2498,  714,  917, 1757,
        537,  806, 2724, 2432,  725, 2683, 1338, 2702, 1322,  144,  153,
       1646,  867, 1612, 1025,  732,  843, 1072,  534, 1307, 1018, 2164,
        418, 2445, 2370, 1953, 2630,   85,  899, 2480,  189, 1034, 1244,
       1241,   46,  175, 2218,  576, 2332, 2267, 2045, 1057, 1442,  897,
        828, 1729, 1837,   98,  350, 1389, 1397,  946,  781, 1906, 1033,
       1365, 1023, 1041, 1007,  878, 1479, 1967, 1027, 2130,   63, 2185,
       1465,  950, 2572,  348,  962,   39,  502,  399, 2595,  512,  690,
        652, 1991,  626,  414, 1763, 1933, 1546,  9

In [122]:
indices2[1]

array([2053,  940, 1440, 2240,  999, 1463,  598, 1814,  183, 2312,  145,
        356, 2344, 1534, 1574,  460, 1941, 1257, 1670,  754, 2025, 1655,
       1492,  412,  608,  850,  423,  165,  863,  175, 1860, 1103,  351,
       1777, 1387, 1593, 2481, 1010, 1135,  372, 1352, 1286, 2475,  988,
       2583, 2711, 2271, 1000,  690,  686,  853,  322,   18, 2704, 2562,
        269,  593, 1335,  875, 2298, 1443,  410,  975, 2283, 2286, 1804,
        478, 1690, 1472, 2251, 1518,  308, 2551, 1347,  494,  630, 1031,
       2571, 1559, 1825, 2680, 2653, 1491,  429, 1503, 2668, 1328, 1255,
       1859, 1368, 1448, 1366,   10,  416, 2154,  436, 2119,  307, 1373,
       1869, 1358,  107, 2301,  683,  966,   20,  512,  218, 2229,  929,
       2428, 1009, 2249,  475, 2494,  676,  195, 2699,  777,  496, 1827,
        155,  679, 2441, 1759, 1644, 1073, 2690, 2280, 2348,   68,  136,
       2729, 2050,  722,  525, 1397, 1750, 1737,  418, 2065,  363, 2104,
       1641, 2713,  440, 2021, 1526,   88, 1272,  5

In [55]:
from os import path
dataset_dir = '../data/zeisel/'
model_save_path = '../data/zeisel/models/'
viz_save_path = '../data/zeisel/visualizations/'

if not path.exists(model_save_path):
  os.makedirs(model_save_path)

if not path.exists(model_save_path + 'experiment_data_folds/'):
  os.makedirs(model_save_path + 'experiment_data_folds/')
  
if not path.exists(viz_save_path):
  os.makedirs(viz_save_path)

adata = sc.read_h5ad(dataset_dir + "Zeisel.h5ad")
X = adata.X.copy()
adata.obs['names']=adata.obs['names0']
adata.obs['annotation'] = adata.obs['names0']
labels = adata.obs['names0'].values
encoder = LabelEncoder()
encoder.fit(labels)
y = encoder.transform(labels)

input_size = X.shape[1]
y

array([2, 2, 2, ..., 1, 1, 1])

In [65]:
_, _, _, train_indices, val_indices, test_indices = split_data_into_dataloaders(X, y, 0.7, 0.1)
train_X_y = adata[np.concatenate([train_indices, val_indices]), :]    

markers = np.load('../data/zeisel/models/all_markers_1.npy', allow_pickle=True)
model = K
train_x = X[np.concatenate([train_indices, val_indices]), :]
train_y = y[np.concatenate([train_indices, val_indices])]
test_x = X[test_indices,:]
test_y = y[test_indices]
    
if markers is not None:
    train_x = train_x[:, markers]
    test_x = test_x[:, markers]

if model is None:
    model = RandomForestClassifier()
model.fit(train_x, train_y)
pred_y = model.predict(test_x)
train_rep = classification_report(train_y, model.predict(train_x), output_dict=True)
test_rep = classification_report(test_y, pred_y, output_dict=True)
cm = confusion_matrix(test_y, pred_y)
if cm is None:
    raise Exception("Some error in generating confusion matrix")
misclass_rate = 1 - accuracy_score(test_y, pred_y)


In [66]:
train_indices

array([1356, 2490, 2466, ..., 2135, 2748, 1673])

In [64]:
np.load('../data/zeisel/models/all_markers_1.npy', allow_pickle=True)

array([   0,    1,    2, ..., 3997, 3998, 3999])